<a href="https://colab.research.google.com/github/kalai2315/NLP_Projects/blob/main/Twitter_Sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NLP_Projects: Twitter Sentimental Analysis**

In [ ]:
  !pip install kaggle

In [ ]:
#configuring the path of kaggle.json file
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
#API to fetch the twitter dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 69% 56.0M/80.9M [00:00<00:00, 208MB/s]
100% 80.9M/80.9M [00:00<00:00, 214MB/s]


In [ ]:
# Extract the zip file
from zipfile import ZipFile
tweetset="/content/sentiment140.zip"
with ZipFile(tweetset, 'r') as zip:
  zip.extractall()
  print("The tweetset is extracted")

The tweetset is extracted


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))#removing these words in order to reduce the complexity of the data

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#load the csv file to pandas dataframe
tweeter_set=pd.read_csv("/content/training.1600000.processed.noemoticon.csv", encoding='ISO-8859-1')

In [ ]:
tweeter_set.shape

(1599999, 6)

In [ ]:
tweeter_set.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
column_names=['target','id','date','flag','user','text']
tweeter_set=pd.read_csv("/content/training.1600000.processed.noemoticon.csv",names=column_names,encoding='ISO-8859-1')

In [ ]:
tweeter_set.shape

(1600000, 6)

In [ ]:
tweeter_set.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
tweeter_set.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
tweeter_set['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [ ]:
#change the target label '4' to '1'
tweeter_set.replace({'target':{4:1}},inplace=True)

In [ ]:
tweeter_set['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [ ]:
port_stem=PorterStemmer

In [ ]:
def stemming(content):
  stemmed_content=re.sub('^a-zA-Z',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [ ]:
tweeter_set['stemmed_content']=tweeter_set['text'].apply(stemming)

KeyboardInterrupt: 

In [ ]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

# Initialize the PorterStemmer
port_stem = PorterStemmer()

# Define a function for stemming
def stemming(text):
    stemmed_content = text.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

# Apply stemming to the 'text' column of your DataFrame
tweeter_set['stemmed_content'] = tweeter_set['text'].apply(stemming)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
tweeter_set.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset can't updat facebook text it... might cr...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,@kenichan dive mani time ball. manag save 50% ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","@nationwideclass no, behav all. i'm mad. here?..."


In [ ]:
print(tweeter_set['stemmed_content'])

0          @switchfoot http://twitpic.com/2y1zl - awww, t...
1          upset can't updat facebook text it... might cr...
2          @kenichan dive mani time ball. manag save 50% ...
3                            whole bodi feel itchi like fire
4          @nationwideclass no, behav all. i'm mad. here?...
                                 ...                        
1599995                       woke up. school best feel ever
1599996    thewdb.com - cool hear old walt interviews! â...
1599997                      readi mojo makeover? ask detail
1599998    happi 38th birthday boo alll time!!! tupac ama...
1599999    happi #charitytuesday @thenspcc @sparkschar @s...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(tweeter_set['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
X=tweeter_set['stemmed_content'].values

In [ ]:
Y=tweeter_set['target'].values

In [ ]:
print(X)

["@switchfoot http://twitpic.com/2y1zl - awww, that' bummer. shoulda got david carr third day it. ;d"
 "upset can't updat facebook text it... might cri result school today also. blah!"
 '@kenichan dive mani time ball. manag save 50% rest go bound' ...
 'readi mojo makeover? ask detail'
 'happi 38th birthday boo alll time!!! tupac amaru shakur'
 'happi #charitytuesday @thenspcc @sparkschar @speakinguph4h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['watch saw iv drink lil wine' "@hatermagazin i'm in!"
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 "hope everyon mother great day! can't wait hear guy store tomorrow!"
 'love wake folgers. bad voic deeper his.']


In [ ]:
print(X_test)

['@mmangen - fine! much time chat twitter. hubbi back summer &amp; tend domin free time.'
 'ah may show w/ ruth kim &amp; geoffrey sanhueza'
 '@ishatara mayb bay area thang dammit!' ...
 '@destini41 nevertheless hooray! 4700 member wonder safe trip!'
 'feel well' '@supersandro !!!! thank you!']


Converting Textual Data to Numerical Data using TFIDF Vectorizer

In [ ]:
vectorizer=TfidfVectorizer()
X_train= vectorizer.fit_transform(X_train)
X_test= vectorizer.fit(X_test)


In [ ]:
print(X_train)

  (0, 555967)	0.448720931098355
  (0, 311959)	0.4213989894780769
  (0, 166830)	0.37842601975344053
  (0, 254267)	0.5248773503338933
  (0, 453116)	0.35795997954657904
  (0, 548734)	0.272460245511403
  (1, 247278)	0.4216951552804088
  (1, 226262)	0.9067376665899747
  (2, 375364)	0.1668911233829414
  (2, 193769)	0.20184466588151023
  (2, 214788)	0.18696286991487276
  (2, 245846)	0.16108271281506448
  (2, 516778)	0.15260594669966604
  (2, 349418)	0.2408559466227688
  (2, 556376)	0.33614580595161764
  (2, 129252)	0.3117944515681846
  (2, 545128)	0.32796751926832934
  (2, 513655)	0.3201716231105769
  (2, 190522)	0.29198833954883197
  (2, 514638)	0.18602125106063971
  (2, 184426)	0.1903000016055046
  (2, 166830)	0.46135196432581554
  (3, 239574)	0.38340980743118147
  (3, 519470)	0.2688525366668751
  (3, 493022)	0.3914375454549087
  :	:
  (1279996, 410084)	0.21029847974488325
  (1279996, 315358)	0.2219434776579764
  (1279996, 378383)	0.1776339096901187
  (1279996, 520651)	0.1868087345439146
  

Training the Machine Learning Model Using Logistic Regression

In [ ]:
model=LogisticRegression(max_iter=1000)
model.fit(X_train,Y_train)


LogisticRegression(max_iter=1000)

Model Evaluation- Accuracy Score

In [ ]:
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(Y_train,X_train_prediction)

In [ ]:
print('Accuracy Score:',training_data_accuracy)

Model Accuracy:

Saving the trained model

In [ ]:
import pickle

In [ ]:
 filename= 'trained_model.sav'
 pickle.dump(model,open(filename,'wb'))

Using the saved model for future predictions

In [ ]:
loaded_model=pickle.load(open('','rb'))

In [ ]:
X_new=X_test[200]
print(Y_test[200])

prediction=model.predict(X_new)
print(prediction)

if (prediction[0]=0):
  print('Negative Tweet')
else:
  print('Positive Tweet')